# (EX) Electronic medical record (as a SQL refresher)
This example utilizes a subset of the [MIMIC III demo](https://physionet.org/content/mimiciii-demo/1.4/) dataset in illustrating the use of database using DuckDB.

*Quick notes about DuckDB:*  

- DuckDB is a relational database for analytics processing (i.e., OLAP)
- DuckDB is columnar-oriented
- DuckDB scales reasonably for *relatively* large datasets and works well for local development

For MIMIC-III, here is a [full summary](https://mit-lcp.github.io/mimic-schema-spy/) where you can understand the relationship between tables.

In [ ]:
!pip install duckdb==1.2.2
import duckdb

In [ ]:
# establish connection
conn = duckdb.connect('dataset/mimic.db', read_only=True)

conn.sql('SHOW TABLES;')

In [ ]:
# metadata for a particular table
conn.sql(
    """
    SELECT column_name, data_type FROM information_schema.columns 
    WHERE table_name = 'PATIENTS';
    """
)

In [ ]:
# admission table at a glance
conn.sql(
    """
    SELECT sum(expire_flag) FROM PATIENTS;
    """
)

In [ ]:
# number of patients
conn.sql(
    """
    SELECT COUNT(*) FROM PATIENTS;
    """
)

## Refresher for simple queries

1. How many records are in each of the available tables?
2. How many patients are female?
3. How many patients passed away during the hospital stay?
4. How many different admission types are there? What are they?
5. What is the earliest and the latest admission time in the database?

In [ ]:
conn.sql(
    """
    SELECT count(*) from PATIENTS
    WHERE gender = 'F'
    """
)

In [ ]:
conn.sql(
    """
    SELECT COUNT(*) FROM PATIENTS
    WHERE dod_hosp IS NOT NULL
    """
)

conn.sql(
    """
    SELECT COUNT(*) FROM ADMISSIONS
    WHERE deathtime IS NOT NULL
    """
)

In [ ]:
conn.sql(
    """
    SELECT DISTINCT admission_type FROM admissions
    """
)

In [ ]:
conn.sql(
    """
    select min(admittime), max(admittime) FROM ADMISSIONS
    """
)

## Slightly more complicated queries
1. Create a table with all ICU stays with their respective patient information.
2. Create a table to show all unique DRG (diagnosis-related group) codes and the number of associated admissions.

In [ ]:
conn.sql(
    """
    SELECT * FROM ICUSTAYS 
    JOIN admissions 
    ON ICUSTAYS.subject_id = admissions.subject_id
    """
)

## Derivative queries

1. Calculate the age of each patient at the time of admission.  
    *Hint:* `cast(patients.dob as date)` allows for addition and subtraction of dates (in days)
2. Identify if a patient passes away when they are in the ICU.
3. Calculate the average duration of admission.

In [ ]:
conn.sql(
    """
    
    """
)

# Creating your own database

The example below refers to creating your own database and loading data into tables.

In [2]:
import duckdb

In [6]:
# establish connection
conn = duckdb.connect('dataset/mimic_new.db', read_only=False)

conn.sql('SHOW TABLES;')

┌──────────┐
│   name   │
│ varchar  │
├──────────┤
│ PENGUINS │
└──────────┘

In [4]:
import pandas as pd

df = pd.read_csv('dataset/penguins.csv')

df.to_sql("PENGUINS", conn, index=False)

C:\Users\moses\AppData\Local\Temp\ipykernel_11824\342381945.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql("PENGUINS", conn, index=False)


-1

In [5]:
conn.close()

In [7]:
conn.sql(
    """
    SELECT * FROM PENGUINS;
    """
)

┌─────────┬───────────┬────────────────┬───────────────┬───────────────────┬─────────────┬─────────┐
│ species │  island   │ bill_length_mm │ bill_depth_mm │ flipper_length_mm │ body_mass_g │   sex   │
│ varchar │  varchar  │     float      │     float     │       float       │    float    │ varchar │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼─────────┤
│ Adelie  │ Torgersen │           39.1 │          18.7 │             181.0 │      3750.0 │ MALE    │
│ Adelie  │ Torgersen │           39.5 │          17.4 │             186.0 │      3800.0 │ FEMALE  │
│ Adelie  │ Torgersen │           40.3 │          18.0 │             195.0 │      3250.0 │ FEMALE  │
│ Adelie  │ Torgersen │           NULL │          NULL │              NULL │        NULL │ NULL    │
│ Adelie  │ Torgersen │           36.7 │          19.3 │             193.0 │      3450.0 │ FEMALE  │
│ Adelie  │ Torgersen │           39.3 │          20.6 │             190.0 │      3650.0 │ 